<!DOCTYPE html>
<html lang="en">
<body>
<h1> PYTHON FINAL PROJECT </h1>
<h3> 2212-INSY-5336-001-PYTHON PROGRAMMING </h3>
<h4>~By Aishwarya M Rao</h4>
</body>
</html>

<!DOCTYPE html>
<html lang="en">
<body>
<h4>1. Write a python program that fetches movie information for the top 500 most popular movies from Metacritics. On this websites, there is an option to show the top movies. 
On Metacritics, it is called “Movies of All Time”
![image.png](attachment:image.png)</h4>
</body>
</html>

In [1]:
#importing all required libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import csv

#creating SSL certificate 
#Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [4]:
#creating 2 empty lists for names of movie and their links 
movie_names = []
movie_link = []
#giving the headers value 
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
#for loop to run through 5 different html pages and get required data 
for i in range(5):  #0-4
    #required url which will loop through 5 times, as 500 movies are required
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)
    print(url)
    #sending the request with url and required headers
    page = requests.get(url, headers = headers)
    print(page)
    #using beautiful soup library to parse the html data
    soup = BeautifulSoup(page.content, "html.parser")
    print(soup)
    #creating a container, with the top most class and table column
    container = soup('td', class_ ="clamp-summary-wrap")
    print(container)
    #writing a for loop to loop through teh container annd get name of movie and their hyperlinks
    for movie in container :
        name = movie.find('h3').text
        hyperlinks =  movie.find('a', class_ = "title")
        link = hyperlinks.get('href', None)
        #appending the links and names to the respective lists 
        movie_names.append(name)
        movie_link.append(link)
#adding the base url, to the list of movie links
for i in range(len(movie_link)):
    movie_link[i] = 'https://www.metacritic.com'+ movie_link[i]
print(len(movie_link))


500


In [5]:
#creating a list
detail_links = []

#for loop to use hyperlink and get detailed links of the movie name 
for link in movie_link:
    #creating page request and html parser
    page = requests.get(link, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup('div', class_= 'see_all pad_btm1')
    
    for line in div:
        #finding the html content with ancor tag 
        hyperlink = line.find('a')
        #getting the url using get function
        url = hyperlink.get('href', None)
        detail_links.append(url)
        #print(url)
print(len(detail_links))

500


In [7]:
#adding the base url to the detailed links of the movies
for i in range(len(detail_links)):
    detail_links[i] = 'https://www.metacritic.com'+ detail_links[i]
print(len(detail_links))

500


In [8]:
import re
#creating empty dictionary
movie_dictionary = dict()
# creating a for loop to loop through the length of the details link 
try:
    for i in range(len(detail_links)):
        #getting the detailed url of the required movie
        url = detail_links[i]
        #creating page request and html parser
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        #finding the div tag and the required class 
        credits = soup.find('div', class_ = 'credits_list')
        #getting the director credits from the table
        director_credits = credits.find('table', class_="credits", summary = re.compile('Director Credits$'))
    
        check_dir = str(director_credits)
        #checking if director credits is null or not 
        if (check_dir != 'None'):
            #finding all values with class as person in the url
            director_line = director_credits.find_all('td', class_ = 'person')
            #creating a list to append the directors
            directors_list = list()
            #a for loop to get director values
            for line in director_line:
                hyperlink = line.find('a') 
                director = hyperlink.get_text()
                directors_list.append(director.strip())
            
                #getting the principle cast and cast credits from wthe web page using regex
                principle_cast_credits = credits.find('table', class_="credits", summary = re.compile('Principal Cast Credits$'))
                other_cast_credits = credits.find('table', class_ = 'credits', summary = re.compile('^(?!.*Principal Cast Credits).*Cast Credits*$'))
                check_ocast = str(other_cast_credits)
                check_pcast = str(principle_cast_credits)
            
            #checking if cast credits is null or not 
            if(check_ocast != 'None' and check_pcast != 'None' and check_pcast != None and check_ocast != None):
                #creating a try and except block to check for errors
                try:
                    other_cast_line = other_cast_credits.find_all('td', class_ = 'person')
                    principle_cast_line = principle_cast_credits.find_all('td', class_ = 'person')
                except :
                    pass
                #creating empty lists to add casts 
                total_cast = list()
                p_cast_list = list()
                o_cast_list = list()
            
                #getting the total cast and principal cast list 
                for oline in other_cast_line:
                    ohyperlink = oline.find('a') 
                    ocast = ohyperlink.get_text()
                    o_cast_list.append(ocast.strip())
                for pline in principle_cast_line:
                    phyperlink = pline.find('a') 
                    pcast = phyperlink.get_text()
                    p_cast_list.append(pcast.strip())
                #adding the cast members
                total_cast = p_cast_list + o_cast_list
                #creating the dictionary
                director_cast= dict()
                #creating dictionary with direcdtor_list as tuple 
                director_cast[tuple(directors_list)] =  total_cast
                #creating a dictionary of moviename, director and cast name
                movie_dictionary[movie_names[i]] = director_cast
            #print(movie_dictionary, '\n')
            #in case of no cast, add null value
            else:
                director_cast_1 = dict()
                cast_1 = list()
                cast_1.append('null')
                director_cast_1[tuple(directors_list)] = cast_1
                movie_dictionary[movie_names[i]] = director_cast_1
        #in case of no director, add null value
        else:
            director_cast_2 = dict()
            directors_2 = list()
            directors_2.append('null') 
            casts_2 = list()
            casts_2.append('null')
            director_cast_2[tuple(directors_2)] = casts_2
            movie_dictionary[movie_names[i]] = director_cast_2
except :
    print('error in retriving the director and cast')

In [26]:
print(directors_list)
#print(movie_dictionary)
type(movie_dictionary)

['Steven Soderbergh']


dict

In [52]:
#appending the dictionary to a list of all_movies 
all_movies = []
all_movies.append(movie_dictionary)
print("All Movies: \n", all_movies)

All Movies: 
 [{'Citizen Kane': {('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, 'The Godfather': {('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, 'Rear Window': {('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, 'Casablanca': {('Michael Curtiz',): ['Humphrey Bogart', 'Ingrid Bergman', 'Clau

In [ ]:
<!DOCTYPE html>
<html lang="en">
<body>
<p>exporting to CSV<p>
</body>
</html>

In [13]:
#importing csv module 
import csv

try:
    #creating a csv document and giving specification such as name of file in write mode
    with open('aishwarya.csv', mode='w', newline='') as movie_file: 
        movie_writer = csv.writer(movie_file) 
        #creating a for loop to loop through each movie dictionary 
        for i in all_movies: 
            #loop through Movie (key) and value (director and cast)
            for key1, value1 in i.items(): 
                #loop through director (key) and cast (value)   
                for key2, value2 in value1.items():  
                    #adding them to the csv file 
                    movie_writer.writerow([key1, key2, value2]) 
    print("Exported CSV file, Successfully!")
except:
    print("csv file error")

Exported CSV file, Successfully!


In [53]:
try:
    while(1) :
        #taking user input 
        input_value = input('Which movie do you want to check? >')
        #asking for what infor user wants to check
        inp = input('What information about this movie do you want to check? (Choose d for director or c for cast) >')
        #creating a for loop to run throu all movies
        for dictionary in all_movies:
            #for loop to loop in the items
            for dict_key, dict_value in dictionary.items():
                #checking for input value equal to dictionary keys 
                if input_value == dict_key:
                    #for director values
                    if inp == 'd':
                        for sub_key, sub_value in dict_value.items():
                            director = ''
                            #getting director value from sub dictionary key 
                            for i in sub_key: 
                                director = director + i + ', '
                            print('The director of the movie', input_value, 'is', director)
                    #for cast values
                    elif inp =='c':
                        for sub_key, sub_value in dict_value.items():
                            cast = ''
                            #getting cast list from sub dictionary values 
                            for j in sub_value:
                                cast = cast + j + ', '
                            print('The cast of the movie', input_value, 'includes', cast)
                #error correction 
                elif input_value != dict_key :
                    print('Make sure you have entered the name of the movie correctly.')
                    break
except:
    print("Please enter correct details.")

Which movie do you want to check? >GHJ
What information about this movie do you want to check? (Choose director or cast) >SAHJ
Make sure you have entered the name of the movie correctly.
Please enter correct details.


In [23]:
try:
    #block of code for writing to get only director and list of cast members 
    list_of_movie_key = []
    for dictionary in all_movies:
        for movie_key, movie_value in dictionary.items():
            #appending the movie name to the list 
            list_of_movie_key.append(movie_key)
            #print(len(movie_key))
            #print(movie_value)
    #creating a list of cast
    sublist_of_movie = list(movie_dictionary.values())
    print(sublist_of_movie)
except:
    print("error in list_of_movie_key")

24
[{('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, {('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, {('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, {('Michael Curtiz',): ['Humphrey Bogart', 'Ingrid Bergman', 'Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wilson', 'Enrique Acosta',

<!DOCTYPE html>
<html lang="en">
<body>
<h4>1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?![image.png](attachment:image.png)</h4>
</body>
</html>

In [15]:
try:
    #creating a tuple and inizialing a count
    director_set = tuple()
    count=0
    #for all count value less than length of movie dictionary the loop will run
    while (count < len(movie_dictionary)):
        #creating a for loop to run through the list and give the count directros
        for i in all_movies:
            #adding the values and count
            sub = list(movie_dictionary.values())[count]
            sub_key = list(sub.keys())
            for values in sub_key:
                #adding director name and count to the tuple
                director_set = director_set+values     
        count = count+1
    #print(director_set)
    #creating a list of all directors with the count 
    all_directors = list(director_set)
    director_count = dict()
    #counting the values in the dictionary 
    for i in all_directors :
        director_count[i] = director_count.get(i, 0) + 1 
    print("The following is name of the director and the number of times each director has appered: ")
    for key in director_count.keys():
        print(key, '->', director_count[key])
except :
    print('error in count')

The following is name of the director and the number of times each director has appered: 
Orson Welles -> 5
Francis Ford Coppola -> 5
Alfred Hitchcock -> 12
Michael Curtiz -> 4
Richard Linklater -> 3
Krzysztof Kieslowski -> 2
Gene Kelly -> 1
Stanley Donen -> 1
Charles Chaplin -> 2
Barry Jenkins -> 2
D.W. Griffith -> 1
Ben Sharpsteen -> 4
Bill Roberts -> 4
Hamilton Luske -> 2
Jack Kinney -> 2
Norman Ferguson -> 3
T. Hee -> 2
Wilfred Jackson -> 4
John Huston -> 5
Guillermo del Toro -> 2
Billy Wilder -> 3
Steve James -> 3
Akira Kurosawa -> 2
Joseph L. Mankiewicz -> 1
François Truffaut -> 2
Sam Peckinpah -> 1
Jim Sheridan -> 1
Carol Reed -> 2
Stanley Kubrick -> 5
Jasmila Zbanic -> 1
George Cukor -> 4
Sam Wood -> 1
Victor Fleming -> 1
Cristian Mungiu -> 1
Sergei M. Eisenstein -> 1
Elia Kazan -> 2
George Lucas -> 2
John Elliotte -> 1
Samuel Armstrong -> 3
Alfonso Cuarón -> 3
Ernst Lubitsch -> 1
Sidney Lumet -> 2
Kenneth Lonergan -> 1
Roman Polanski -> 2
Steve McQueen -> 4
Charles Burnett -> 

In [16]:
try:
    #creating a empty cast list 
    cast_members = list()
    count=0
    #for all count value less than length of movie dictionary the loop will run
    while (count < len(movie_dictionary)):
        for i in all_movies:
            #taking the values of the sub dictionary with count
            sub = list(movie_dictionary.values())[count] 
            sub_values = list(sub.values())
            for values in sub_values:
                #adding values to the cast members list
                cast_members = cast_members + values
        #incrementing count
        count = count+1
    #print(all_cast_members)  
    #cretaing a list of all cast members
    all_cast_members = list(cast_members)
    total_cast_count = dict()
    for i in all_cast_members :
        #counting the cast members have appered in the dictionary
        total_cast_count[i] = total_cast_count.get(i, 0) + 1 
    print("The following is the number of times each cast member has appered: ")
    #printing the same
    for key in total_cast_count.keys():
        print(key, '->', total_cast_count[key])
except:
    print('error in the cast counting')

The following is the number of times each cast member has appered: 
Joseph Cotten -> 5
Orson Welles -> 6
Agnes Moorehead -> 2
Dorothy Comingore -> 1
Erskine Sanford -> 2
Everett Sloane -> 1
Fortunio Bonanova -> 2
George Coulouris -> 1
Georgia Backus -> 3
Gus Schilling -> 1
Harry Shannon -> 2
Paul Stewart -> 2
Philip Van Zandt -> 1
Ray Collins -> 4
Ruth Warrick -> 1
William Alland -> 1
Al Pacino -> 4
Marlon Brando -> 7
Abe Vigoda -> 1
Al Lettieri -> 1
Diane Keaton -> 3
Gianni Russo -> 1
James Caan -> 3
John Cazale -> 2
John Marley -> 2
Richard Conte -> 1
Richard S. Castellano -> 1
Robert Duvall -> 5
Rudy Bond -> 4
Sterling Hayden -> 4
Talia Shire -> 2
Frank Cady -> 1
Georgine Darcy -> 1
Grace Kelly -> 2
James Stewart -> 6
Judith Evelyn -> 1
Raymond Burr -> 1
Ross Bagdasarian -> 1
Sara Berner -> 1
Thelma Ritter -> 2
Wendell Corey -> 1
Havis Davenport -> 1
Irene Winston -> 1
Jerry Antes -> 1
Jesslyn Fax -> 1
Rand Harper -> 1
Humphrey Bogart -> 5
Ingrid Bergman -> 2
Claude Rains -> 3
Conra

Alexander James-Blake -> 1
Frankie Fox -> 1
Jermaine Freeman -> 1
Romario Simpson -> 1
Saffron Coomber -> 1
Tim Allen -> 3
Tom Hanks -> 4
Annie Potts -> 2
Don Rickles -> 3
Erik von Detten -> 1
Jack Angel -> 1
Jim Varney -> 2
John Morris -> 4
Laurie Metcalf -> 4
Penn Jillette -> 1
R. Lee Ermey -> 1
Sarah Freeman -> 1
Spencer Aste -> 1
Wallace Shawn -> 5
Boris Karloff -> 3
Colin Clive -> 2
Douglas Walton -> 1
E.E. Clive -> 2
Elsa Lanchester -> 3
Ernest Thesiger -> 1
Gavin Gordon -> 1
Lucien Prival -> 1
Una O'Connor -> 3
Valerie Hobson -> 2
Anne Darling -> 1
Dwight Frye -> 2
Mary Gordon -> 1
O.P. Heggie -> 1
Reginald Barlow -> 1
Andrew Garfield -> 1
Jesse Eisenberg -> 1
Justin Timberlake -> 3
Adina Porter -> 1
Alecia Svensen -> 1
Armie Hammer -> 3
Brenda Song -> 1
Bryan Barter -> 1
Caleb Landry Jones -> 5
Calvin Dean -> 1
Cherilyn Wilson -> 1
Dakota Johnson -> 1
Denise Grayson -> 1
Dustin Fitzsimons -> 1
James Dastoli -> 1
Jami Owen -> 1
Jayk Gallagher -> 1
Joseph Mazzello -> 1
Malese Jow

Robert Smigel -> 2
Scarlett Johansson -> 5
Annie Hamilton -> 1
Jasmine Cephas Jones -> 2
Martha Kelly -> 1
Mary Wiseman -> 1
Motell Gyn Foster -> 1
Pete Simpson -> 1
Raymond J. Lee -> 1
Emma Stone -> 5
J.K. Simmons -> 4
Rosemarie DeWitt -> 2
Ryan Gosling -> 2
Amanda Fields -> 2
Amiée Conn -> 1
Ana Flavia Gavlak -> 1
Andres Perez-Molina -> 1
Ashley Caple -> 1
Callie Hernandez -> 1
Cameron Brinkman -> 1
Candice Coke -> 1
Christopher Aber -> 1
Cinda Adams -> 1
Claudine Claudio -> 1
D.A. Wallach -> 1
Finn Wittrock -> 3
Hemky Madera -> 1
Jason Fuchs -> 1
Jessica Rothe -> 1
John Legend -> 1
Jordan Ray Fox -> 1
Josh Pence -> 1
Kaye L. Morris -> 1
Kc Monnie -> 1
Kiff VandenHeuvel -> 1
Kristin Slaysman -> 1
Margaret Newborn -> 1
Meagen Fay -> 1
Miles Anderson -> 1
Morgan Larson -> 1
Nilla Watkins -> 1
Phillip E. Walker -> 1
Sandra Rosko -> 1
Shannon Klein -> 1
Shannon Leann -> 1
Sonoya Mizuno -> 1
Terry Walters -> 1
Thom Shelton -> 1
Tom Everett Scott -> 1
Trevor Lissauer -> 1
Zoë Hall -> 1
Agn

Amy Wright -> 1
Barbara Barrie -> 1
Daniel Stern -> 1
David K. Blase -> 1
Hart Bochner -> 1
Jackie Earle Haley -> 2
Jennifer K. Mickel -> 1
John Ashton -> 1
Lisa Shure -> 1
P.J. Soles -> 2
Paul Dooley -> 1
Peter Maloney -> 1
Robyn Douglass -> 1
Alice McMillan -> 2
Chyna Terrelonge-Vaughan -> 2
Fabrizio Matteini -> 2
Frankie Wilson -> 1
Hannah Ashby Ward -> 2
Honor Swinton-Byrne -> 1
Jack McMullen -> 1
Jake Phillips Head -> 2
James Spencer Ashworth -> 2
Janet Etuk -> 2
Jaygann Ayeh -> 1
Neil Young -> 1
Richard Ayoade -> 3
Tilda Swinton -> 2
Tom Burke -> 1
Tosin Cole -> 1
Barbara Peirson -> 1
Crispin Buxton -> 1
Dick Fontaine -> 1
Jack W. Gregory -> 1
James Barrett -> 1
James Dodds -> 1
Jaz Dalrymple -> 1
Leonardo Bozzo -> 1
Richard Tree -> 1
Steve Gough -> 1
Géza Röhrig -> 2
Levente Molnár -> 2
Todd Charmont -> 2
Urs Rechn -> 2
Amitai Kedar -> 1
Attila Fritz -> 1
Balázs Farkas -> 1
Björn Freiberg -> 1
Christian Harting -> 1
Eszter Csépai -> 1
Gergö Farkas -> 1
Jerzy Walczak -> 1
Juli Ja

Gordon B. Clarke -> 1
Vincent Gardenia -> 1
Kevin Spacey -> 1
Kim Basinger -> 1
Russell Crowe -> 1
Danny DeVito -> 1
Elisabeth Granli -> 1
Graham Beckel -> 1
Matt McCoy -> 1
Paolo Seganti -> 1
Ron Rifkin -> 2
Sandra Taylor -> 1
Steve Rankin -> 1
James Nesbitt -> 1
Nicholas Farrell -> 1
Tim Pigott-Smith -> 1
Allan Gildea -> 1
Carmel McCallion -> 1
Christopher Villiers -> 1
Declan Duddy -> 1
Edel Frazer -> 1
Gerard Crossan -> 1
Gerry Hammond -> 1
James Hewitt -> 1
Jason Stammers -> 1
Joanne Lindsay -> 1
Mary Moulds -> 1
Mike Edwards -> 1
Bern Collaco -> 1
Camilla Rutherford -> 3
Cedric Tylleman -> 2
Dave Simon -> 2
Dinah Nicholson -> 1
Elli Banks -> 1
Harriet Leitch -> 2
Ingrid Sophie Schram -> 2
Jack Gordon -> 1
Jane Perry -> 2
Jason Redshaw -> 2
Joan Brown -> 1
Julie Duck -> 1
Kelly Schembri -> 2
Maryanne Frost -> 1
Pip Phillips -> 2
Richard Graham -> 2
Sarah Lamesch -> 2
Sue Clark -> 1
Vicky Krieps -> 2
Alexandra Moloney -> 1
Amber Brabant -> 1
Amy Cunningham -> 1
Andrew G. Ogleby -> 

Wilhelmina Blanchard -> 1
Douglas Brinkley -> 1
Joseph Bruno -> 1
Karen Carter -> 1
Kathleen Blanco -> 1
Louis Cataldie -> 1
Ken Davitian -> 1
Sacha Baron Cohen -> 1
Alan Keyes -> 1
Bob Barr -> 1
Jean-Pierre Parent -> 1
Luenell -> 1
Mitchell Falk -> 1
Pamela Anderson -> 1
Alan S. Kim -> 1
Darryl Cox -> 1
Eric Starkey -> 1
Esther Moon -> 1
Noel Cho -> 1
Scott Haze -> 1
Ye-ri Han -> 1
Yeo-Jong Yun -> 1
Ben Hall -> 1
Jacob M Wade -> 1
James Carroll -> 1
Jenny Phagan -> 1
Marisa Berenson -> 1
Ryan O'Neal -> 1
André Morell -> 2
Arthur O'Sullivan -> 1
Diana Körner -> 1
Frank Middlemass -> 1
Gay Hamilton -> 1
Godfrey Quigley -> 1
Hardy Krüger -> 1
Leonard Rossiter -> 1
Marie Kean -> 1
Murray Melvin -> 1
Philip Stone -> 1
Steven Berkoff -> 1
Aoi Mizuhara -> 2
Awkwafina -> 1
Becca Khalil -> 2
Diana Lin -> 1
Diane Dehn -> 2
Gil Perez-Abraham -> 1
Han Chen -> 2
Hong Lin -> 1
Hong Lu -> 2
Ines Laimins -> 2
Jim Liu -> 1
Shuzhen Zhou -> 1
Tzi Ma -> 1
X Mayo -> 2
Yang Xuejian -> 1
Yongbo Jiang -> 2
Z

Stacey LaBerge -> 1
Thomas Hauff -> 1
Tom Harvey -> 1
Wendy Crewson -> 2
Michèle Morgan -> 1
Bobby Henrey -> 1
Dandy Nichols -> 1
Gerard Heinz -> 1
Jack Hawkins -> 2
James Hayter -> 1
Joan Young -> 1
Karel Stepanek -> 1
Sonia Dresdel -> 1
Torin Thatcher -> 1
Art Aragon -> 1
Billy Walker -> 1
Curtis Cokes -> 1
Sixto Rodriguez -> 1
Stacy Keach -> 3
Susan Tyrrell -> 1
Wayne Mahan -> 1
Al Silvani -> 1
Álvaro López -> 1
Bill Riddle -> 1
Carl D. Parker -> 1
Ruben Navarro -> 1
Aryeh Cherner -> 2
Dekel Adin -> 1
Gefen Barkai -> 1
Lior Ashkenazi -> 1
Rami Buzaglo -> 1
Ran Buxenbaum -> 1
Sarah Adler -> 1
Shaul Amir -> 1
Shira Haas -> 1
Yehuda Almagor -> 1
Yonaton Shiray -> 1
Eden Daniel -> 1
Eden Gmliel -> 1
Ilia Grosz -> 1
Itay Exlroad -> 1
Yael Eisenberg -> 1
Anthony Hopkins -> 2
Emma Thompson -> 2
Adrian Ross Magenty -> 1
Helena Bonham Carter -> 3
Ian Latimer -> 1
Jemma Redgrave -> 2
Jo Kendall -> 1
Joseph Bennett -> 1
Mary Nash -> 1
Prunella Scales -> 1
Samuel West -> 1
Siegbert Prawer -> 1


Alex Wolff -> 1
Ann Dowd -> 2
Austin R. Grant -> 2
BriAnn Rachele -> 1
Christy Summerhays -> 1
Gabriel Monroe Eckert -> 1
Harrison Nell -> 1
Jake Brown -> 1
Jarrod Phillips -> 2
John Forker -> 2
Mallory Bechtel -> 2
Marilyn Miller -> 2
Mark Blockovich -> 1
Milly Shapiro -> 1
Morgan Lund -> 1
Rachelle Hardy -> 2
Toni Collette -> 1
Zachary Arthur -> 2
A.J. Moss -> 1
Brock McKinney -> 1
Chad Wright -> 1
Gerry Garcia -> 1
Heidi Méndez -> 1
Jason Miyagi -> 1
Lorenzo Silva -> 1
Moises L. Tovar -> 1
Shane Morrisun -> 1
John Randolph -> 1
Albert Henderson -> 1
Barbara Eda-Young -> 1
Biff McGuire -> 1
Cornelia Sharpe -> 1
Damien Leake -> 1
Edward Grover -> 1
Hank Garrett -> 1
Jack Kehoe -> 1
John Medici -> 1
Joseph Bova -> 1
Norman Ornellas -> 1
Catalina Sandino Moreno -> 1
Guilied Lopez -> 1
Charles Albert Patiño -> 1
Evangelina Morales -> 1
Fabricio Suarez -> 1
Jaime Osorio Gómez -> 1
Johanna Andrea Mora -> 1
John Álex Toro -> 1
Juana Guarderas -> 1
Mateo Suarez -> 1
Rodrigo Sánchez Borhorque

<!DOCTYPE html>
<html lang="en">
<body>
<h3> Cosine values </h3>
<h4>Q2. Create a dictionary of actors/actresses that the directors have worked together with in each movie, then calculate their cosine similarity, which directors work with similar groups of actors/actresses? Use director name as the dictionary name, actor/actress name as the key, and the times they have worked together in a movie as the value. ![image.png](attachment:image.png) </h4>
</body>
</html>

In [17]:
try:
    #printing all director names
    for keys in director_count.keys():
        print(keys)

    #taking input from user
    print('enter two directors names from the above list: ')
    director1 = input("enter 1st directors name: ")
    director2 = input("enter 2nd directors name: ")
except:
    print('enter correct inputs')

Orson Welles
Francis Ford Coppola
Alfred Hitchcock
Michael Curtiz
Richard Linklater
Krzysztof Kieslowski
Gene Kelly
Stanley Donen
Charles Chaplin
Barry Jenkins
D.W. Griffith
Ben Sharpsteen
Bill Roberts
Hamilton Luske
Jack Kinney
Norman Ferguson
T. Hee
Wilfred Jackson
John Huston
Guillermo del Toro
Billy Wilder
Steve James
Akira Kurosawa
Joseph L. Mankiewicz
François Truffaut
Sam Peckinpah
Jim Sheridan
Carol Reed
Stanley Kubrick
Jasmila Zbanic
George Cukor
Sam Wood
Victor Fleming
Cristian Mungiu
Sergei M. Eisenstein
Elia Kazan
George Lucas
John Elliotte
Samuel Armstrong
Alfonso Cuarón
Ernst Lubitsch
Sidney Lumet
Kenneth Lonergan
Roman Polanski
Steve McQueen
Charles Burnett
Robert Altman
Brad Bird
Joon-ho Bong
Sarah Gavron
Nicolas Roeg
John Ford
Marcel Carné
Martin Scorsese
Preston Sturges
David Hand
Ford Beebe Jr.
James Algar
Jim Handley
Paul Satterfield
Hayao Miyazaki
Gary Trousdale
Kirk Wise
John Lasseter
James Whale
David Fincher
Céline Sciamma
Raoul Peck
Michael Wadleigh
Larry Morey

In [104]:
try:
    #creating empty directors dictionary
    directors1 = {}
    #first loop to loop through the sublist of movie list 
    for all_movie in sublist_of_movie:
        #second loop to loop through items of all_movie
        for key1,val1 in all_movie.items():
            #check for x in key value of all_movie.items()
            for x in key1:
                #if its not there, then initialize a dictionary with x as key
                if x not in directors1:
                    directors1[x]= {}
                    #print(directors1)
                #for values in the value from all_movie.items()
                for v in val1:
                    #in the value is not present, then initialize count as 1
                    if v not in directors1[x]:
                        directors1[x][v]=1
                    else:
                        #if the value is already present, then increment count by 1
                        directors1[x][v]+=1
    #print(directors1['Orson Welles'])
except:
    print("error in calculating...")

{'Joseph Cotten': 3, 'Orson Welles': 5, 'Agnes Moorehead': 2, 'Dorothy Comingore': 1, 'Erskine Sanford': 2, 'Everett Sloane': 1, 'Fortunio Bonanova': 1, 'George Coulouris': 1, 'Georgia Backus': 2, 'Gus Schilling': 1, 'Harry Shannon': 1, 'Paul Stewart': 2, 'Philip Van Zandt': 1, 'Ray Collins': 3, 'Ruth Warrick': 1, 'William Alland': 1, 'Akim Tamiroff': 1, 'Charlton Heston': 1, 'Dennis Weaver': 1, 'Janet Leigh': 1, 'Joanna Moore': 1, 'Joseph Calleia': 1, 'Mort Mills': 1, 'Valentin de Vargas': 1, 'Joi Lansing': 1, 'Lalo Rios': 1, 'Michael Sargent': 1, 'Phil Harvey': 1, 'Victor Millan': 1, 'Andrés Mejuto': 1, 'Jeanne Moreau': 1, 'John Gielgud': 1, 'Julio Peña': 1, 'Margaret Rutherford': 1, 'Marina Vlady': 1, 'Michael Aldridge': 1, 'Tony Beckley': 1, 'Walter Chiari': 1, 'Alan Webb': 1, 'Fernando Rey': 1, 'Jeremy Rowe': 1, 'Keith Baxter': 1, 'Keith Pyott': 1, 'Anne Baxter': 1, 'Dolores Costello': 1, 'Edwin August': 1, 'Richard Bennett': 1, 'Tim Holt': 1, 'Harry A. Bailey': 1, 'Jack Baxley': 

In [65]:
try:
#we first need to extract the list of actors and how many time they have worked together
    if director1 in director_count and director2 in director_count :
        print("valid")
    else:
        print("enter valid director names form the list")
    #creating two dictionaries
    dict_dir_1 = dict()
    dict_dir_2 = dict()
    
    #assigning values to a variable
    val1 = directors1[director1]
    val2 = directors1[director2]

    #initializing the dictionary with director as key and counted values of cast as values 
    dict_dir_1[director1] = val1
    dict_dir_2[director2] = val2
    
    #creating a sub_actors list with the keys of 
    sub_actor1 = list(val1.keys())
    sub_actor2 = list(val2.keys())

    #then we convert them into sets, so that we can find a combined set using the union function
    set_actor1 = set(sub_actor1)
    set_actor2 = set(sub_actor2)
    
    #comine both the sets
    combine_set = set_actor1.union(set_actor2)
    
    print(combine_set)
except:
    print("error")

valid
{'Orson Welles': {'Joseph Cotten': 3, 'Orson Welles': 5, 'Agnes Moorehead': 2, 'Dorothy Comingore': 1, 'Erskine Sanford': 2, 'Everett Sloane': 1, 'Fortunio Bonanova': 1, 'George Coulouris': 1, 'Georgia Backus': 2, 'Gus Schilling': 1, 'Harry Shannon': 1, 'Paul Stewart': 2, 'Philip Van Zandt': 1, 'Ray Collins': 3, 'Ruth Warrick': 1, 'William Alland': 1, 'Akim Tamiroff': 1, 'Charlton Heston': 1, 'Dennis Weaver': 1, 'Janet Leigh': 1, 'Joanna Moore': 1, 'Joseph Calleia': 1, 'Mort Mills': 1, 'Valentin de Vargas': 1, 'Joi Lansing': 1, 'Lalo Rios': 1, 'Michael Sargent': 1, 'Phil Harvey': 1, 'Victor Millan': 1, 'Andrés Mejuto': 1, 'Jeanne Moreau': 1, 'John Gielgud': 1, 'Julio Peña': 1, 'Margaret Rutherford': 1, 'Marina Vlady': 1, 'Michael Aldridge': 1, 'Tony Beckley': 1, 'Walter Chiari': 1, 'Alan Webb': 1, 'Fernando Rey': 1, 'Jeremy Rowe': 1, 'Keith Baxter': 1, 'Keith Pyott': 1, 'Anne Baxter': 1, 'Dolores Costello': 1, 'Edwin August': 1, 'Richard Bennett': 1, 'Tim Holt': 1, 'Harry A. Bail

In [67]:
try:
    #creating a dictionary out of the combined list and filling in 0 for all the values
    dir1_actor = dict()
    #for director1
    for i in combine_set:
        dir1_actor[i] = 0
    print(dir1_actor)

    dir2_actor = dict()
    #for director2
    for i in combine_set:
        dir2_actor[i] = 0
    print(dir2_actor)
except :
    print("error.")

{'Richard Wilson': 0, 'John Gielgud': 0, 'Philip Van Zandt': 0, 'Phil Harvey': 0, 'Steve Cochran': 0, 'Gladys George': 0, 'Laurence Harvey': 0, 'Hoagy Carmichael': 0, 'Alexander Welles': 0, 'Gus Schilling': 0, 'Ruth Warrick': 0, 'Orson Welles': 0, 'Alan Webb': 0, 'Joi Lansing': 0, 'Charlton Heston': 0, 'Jeanne Moreau': 0, 'Olive Ball': 0, 'Joseph Cotten': 0, 'Teresa Wright': 0, 'Jack Baxley': 0, 'Clifford Irving': 0, 'Dolores Costello': 0, 'Edith Irving': 0, 'Elmyr de Hory': 0, 'Harry A. Bailey': 0, 'George Coulouris': 0, 'Keith Baxter': 0, 'Gary Graver': 0, 'Walter Baldwin': 0, 'Walter Chiari': 0, 'Joanna Moore': 0, 'Janet Leigh': 0, 'Margaret Rutherford': 0, 'Harold Russell': 0, 'William Blees': 0, 'Georgia Backus': 0, 'Richard Bennett': 0, 'Howard Hughes': 0, 'Andrés Mejuto': 0, 'Akim Tamiroff': 0, 'Oja Kodar': 0, 'Minna Gombell': 0, 'Mort Mills': 0, 'Jeremy Rowe': 0, 'Valentin de Vargas': 0, 'Dana Andrews': 0, 'Victor Millan': 0, 'Agnes Moorehead': 0, 'Richard Drewett': 0, 'Roman B

In [69]:
try:
    #for those actors that director1 have worked with, 
    #replace the 0 with the actual values from the list 
    for key, value in dir1_actor.items():
        #looping through value items
        for k, v in val1.items():
            if key == k:
                dir1_actor[key] = v
    print(dir1_actor)
    #do the same thing for director2
    for key, value in dir2_actor.items():
        #looping through value items
        for k, v in val2.items():
            if key == k:
                dir2_actor[key] = v
    print(dir2_actor)
except:
    print('error')

{'Richard Wilson': 1, 'John Gielgud': 1, 'Philip Van Zandt': 1, 'Phil Harvey': 1, 'Steve Cochran': 0, 'Gladys George': 0, 'Laurence Harvey': 1, 'Hoagy Carmichael': 0, 'Alexander Welles': 1, 'Gus Schilling': 1, 'Ruth Warrick': 1, 'Orson Welles': 5, 'Alan Webb': 1, 'Joi Lansing': 1, 'Charlton Heston': 1, 'Jeanne Moreau': 1, 'Olive Ball': 1, 'Joseph Cotten': 3, 'Teresa Wright': 0, 'Jack Baxley': 1, 'Clifford Irving': 1, 'Dolores Costello': 1, 'Edith Irving': 1, 'Elmyr de Hory': 1, 'Harry A. Bailey': 1, 'George Coulouris': 1, 'Keith Baxter': 1, 'Gary Graver': 1, 'Walter Baldwin': 0, 'Walter Chiari': 1, 'Joanna Moore': 1, 'Janet Leigh': 1, 'Margaret Rutherford': 1, 'Harold Russell': 0, 'William Blees': 1, 'Georgia Backus': 2, 'Richard Bennett': 1, 'Howard Hughes': 1, 'Andrés Mejuto': 1, 'Akim Tamiroff': 1, 'Oja Kodar': 1, 'Minna Gombell': 0, 'Mort Mills': 1, 'Jeremy Rowe': 1, 'Valentin de Vargas': 1, 'Dana Andrews': 0, 'Victor Millan': 1, 'Agnes Moorehead': 2, 'Richard Drewett': 1, 'Roman B

In [72]:
try:
    #now that we have a uniform dictionary across the 2 directors, 
    #we can extract the values in vectors
    dir1_vector = list(dir1_actor.values())
    dir2_vector = list(dir2_actor.values())
    print(dir1_vector)
    print(dir2_vector)
except:
    print('error in vector')

[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 5, 1, 1, 1, 1, 1, 3, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 3, 1, 2, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]


In [74]:
try:
    #calculate cosine similarity
    divident = 0 
    #multipling vector values
    for i in range(len(dir1_vector)):
        summing = dir1_vector[i] * dir2_vector[i]
        divident = divident + summing
    #print(divident)
    a = 0
    b = 0
    for i in range(len(dir1_vector)):
        #caluclating sum of squares of the vector
        sum1 = (dir1_vector[i])**2
        a = a + sum1
        sum2 = (dir2_vector[i])**2
        b = b + sum2
    divisor = (a**0.5)*(b**0.5)
    #print(divisor)
    #final step of cosine_similarity
    cosine_similarity = divident/divisor
    print(cosine_similarity)
except:
    print('error')

3


<!DOCTYPE html>
<html lang="en">
<body>
<h4> Q3. Pick 5 of your favorite actors/actresses from this list of top 500 movies. Then create a dictionary of all the actors/actresses that they have collaborated with in a movie. Following similar method as above in task 2, create the dictionaries, and compare these 5 actors, who is the most popular supporting actor/actress among them all!</h4>
</body>
</html> 

In [203]:
try:
    #selecting 5 actor values
    actor1 = 'William Alland' 
    actor2 = 'Walter Chiari'
    actor3 = 'Fortunio Bonanova'
    actor4 = 'Cindy Williams'
    actor5 = 'Cory Michael Smith'

    #creating actor list
    actor_list =[actor1,actor2,actor3,actor4,actor5]
    actors_dict = dict()
     #first loop to loop through the sublist of movie list 
    for actor in sublist_of_movie:
        #second loop to loop through items of actor
        for key1,val1 in actor.items():
            #check for v in values in items of actor
            for v in val1:
                #check if the value is present in dict or not 
                if v not in actors_dict:
                    #creating a dictionary of actors with values from above as key
                    actors_dict[v] = {}
                for v1 in val1:
                    if v != v1 :
                        #if the value is not present, then initialize count as 1
                        if v1 not in actors_dict[v]:
                            actors_dict[v][v1]=1
                        #if the value is already present, then increment count by 1
                        else:
                            actors_dict[v][v1]+=1
    #initializing 5 dictionaries
    dict_act_1 = dict()
    dict_act_2 = dict()
    dict_act_3 = dict()
    dict_act_4 = dict()
    dict_act_5 = dict()

    #making a list of these dictionaries
    dict_act_list = [dict_act_1,dict_act_2,dict_act_3,dict_act_4,dict_act_5]

    #then we convert them keys of list of actors into sets, 
    #so that we can find a combined set using the union function
    set_actor_1 = set(list(actors_dict[actor1].keys()))
    set_actor_2 = set(list(actors_dict[actor2].keys()))
    set_actor_3 = set(list(actors_dict[actor3].keys()))
    set_actor_4 = set(list(actors_dict[actor4].keys()))
    set_actor_5 = set(list(actors_dict[actor5].keys()))
    
    #combining the set
    combine_set = set_actor_1.union(set_actor_2).union(set_actor_3).union(set_actor_4).union(set_actor_5)

    #print(combine_set)

    #creating a dictionary out of the combined list, 
    #and filling in 0 for all the values
    for i in combine_set:
        dict_act_1[i] = 0
        dict_act_2[i] = 0
        dict_act_3[i] = 0
        dict_act_4[i] = 0
        dict_act_5[i] = 0
except :
    print("error")

In [202]:
try:
    #for those all those actors that have worked with each other, 
    #replace the 0 with the actual values from the list 
    for key, value in dict_act_1.items():
        for i in range(0,len(actor_list)):
            for k,v in actors_dict[actor_list[i]].items() :
                if key == k:
                    dict_act_list[i][key] = v

    #creating vectors
    act1_vector = list(dict_act_1.values())
    act2_vector = list(dict_act_2.values())
    act3_vector = list(dict_act_3.values())
    act4_vector = list(dict_act_4.values())
    act5_vector = list(dict_act_5.values())

except("error in vector creation")

0


In [199]:
from sklearn.metrics.pairwise import cosine_similarity 
try:
    #creating list of vectors
    list_of_vectors=[act1_vector,act2_vector, act3_vector,act4_vector,act5_vector]
    cosine_sim=0
    co_sin_values = []
    dict_cosine ={}
    #caluclating the cosine similarity of each actor with other actor
    for i in range(len(list_of_vectors)) :
        for j in range(i+1,len(list_of_vectors)) :
            if i != j :
                cosine_sim = cosine_similarity([list_of_vectors[i]],[list_of_vectors[j]])
                co_sin_values.append(float(cosine_sim))
                #creating a dictionary of the same
                dict_cosine[float(cosine_sim)] = [actor_list[i],actor_list[j]]
    #print(co_sin_values)
    #print(dict_cosine)
except:
    print('error')

In [200]:
try:
    #sorting the list
    co_sin_values.sort()

    #the highest vale is most popular pair
    f= dict_cosine[co_sin_values[len(co_sin_values)-1]]

    print("most popular supporting actor/actress among them all is:", f)
except:
    print('error')

most popular supporting actor/actress among them all is: ['William Alland', 'Fortunio Bonanova']


Refrences used : 

1. pdf notes and .ipynb files by Ruochen Liao posted on canvas
2. https://docs.python.org/3/library/index.html
3. https://docs.python.org/3/tutorial/introduction.html#lists
4. https://docs.python.org/3/py-modindex.html
5. https://bugs.python.org/file47781/Tutorial_EDIT.pdf
6. https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=0 to 4